In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the parameters for the strategy
ema_short_period = 12
ema_long_period = 26
signal_period = 9
target_profit = 0.05  # Target profit percentage
stop_loss = 0.03  # Stop loss percentage

# Fetch historical price data using Yahoo Finance API (you can replace 'AAPL' with your desired stock symbol)
symbol = "RELIANCE.NS"
data = yf.download(symbol, start="2013-07-10", end="2023-07-10")

# Calculate MACD indicators
data['ema_short'] = data['Close'].ewm(span=ema_short_period, adjust=False).mean()
data['ema_long'] = data['Close'].ewm(span=ema_long_period, adjust=False).mean()
data['macd_line'] = data['ema_short'] - data['ema_long']
data['signal_line'] = data['macd_line'].ewm(span=signal_period, adjust=False).mean()
data['histogram'] = data['macd_line'] - data['signal_line']

# Detect Double Bottom pattern
data['previous_low'] = data['Low'].shift(1)
data['next_low'] = data['Low'].shift(-1)
data['is_double_bottom'] = np.where(
    (data['Low'] < data['previous_low']) & (data['Low'] < data['next_low']), True, False
)

# Initialize variables
is_in_position = False
buy_price = 0
portfolio_value = 10000  # Initial portfolio value in USD
shares = 0
cumulative_returns = [0]
maximum_drawdown = 0
equity_curve = [portfolio_value]

# Execute the trading strategy
for i in range(1, len(data)):
    close_price = data['Close'][i]
    signal_line = data['signal_line'][i]
    histogram = data['histogram'][i]
    is_double_bottom = data['is_double_bottom'][i]

    previous_portfolio_value = portfolio_value

    if not is_in_position and is_double_bottom and histogram > 0 and signal_line > 0 and histogram > signal_line:
        # Enter long position
        is_in_position = True
        buy_price = close_price
        shares = portfolio_value / buy_price

    elif is_in_position and (histogram < 0 or signal_line < 0 or close_price >= buy_price * (1 + target_profit) or close_price <= buy_price * (1 - stop_loss)):
        # Exit long position
        is_in_position = False
        sell_price = close_price
        portfolio_value = shares * sell_price
        returns = (portfolio_value - previous_portfolio_value) / previous_portfolio_value
        cumulative_returns.append(cumulative_returns[-1] + returns)
        equity_curve.append(portfolio_value)

        if portfolio_value < previous_portfolio_value and portfolio_value - previous_portfolio_value < maximum_drawdown:
            maximum_drawdown = portfolio_value - previous_portfolio_value

# Calculate evaluation metrics
returns = np.array(cumulative_returns)
total_returns = returns[-1]
sharpe_ratio = np.sqrt(252) * np.mean(returns) / np.std(returns)

# Print evaluation metrics
print("Total Returns: {:.2%}".format(total_returns))
print("Maximum Drawdown: {:.2%}".format(maximum_drawdown))
print("Sharpe Ratio: {:.2f}".format(sharpe_ratio))


[*********************100%***********************]  1 of 1 completed
Total Returns: 12.63%
Maximum Drawdown: -41717.42%
Sharpe Ratio: 4.96
